In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np
import pickle
import random
import seaborn as sns
import sys

sys.path.insert(1, '../src')

In [ ]:
from azran_ghahramani import multiscale_k_prototypes
from azran_ghahramani import K_prototypes
from azran_ghahramani import make_distances
from azran_ghahramani import make_W_D
from azran_ghahramani import maximal_eigengap
from azran_ghahramani import star_shaped_init
from azran_ghahramani import get_eigengap_values
from azran_ghahramani import get_maximal_eigengap_information
from kernels import make_laplacian_kernel
from metrics import euclid_distance
from similarities import inverse
from similarities import inverse_squared
from similarities import make_gaussian_similarity
from similarities import make_gaussian_similarity_from_percentile
from utils import make_colours

# Example 1

In [ ]:
def similarity(x):
    if x == 0:
        return 0
    return 1 / (x**3)

metric = euclid_distance

In [ ]:
points_filename = '../data/gaussian_clouds_1.pkl'

In [ ]:
with open(points_filename, 'rb') as fp:
    points = pickle.load(fp)

In [ ]:
x_coords = [el[0] for el in points]
y_coords = [el[1] for el in points]

In [ ]:
sns.set_style("darkgrid", {"axes.facecolor": ".9"})

In [ ]:
sns.scatterplot(x_coords, y_coords, color='purple')

ax = plt.gca()
ax.axes.xaxis.set_visible(False)
ax.axes.yaxis.set_visible(False)

plt.title('Gaussian Clouds Example 1\nUnclustered Points')
plt.show()

In [ ]:
W, D = make_W_D(points, metric, similarity)
transition_matrix = np.linalg.inv(D).dot(W)

evalues, evectors = np.linalg.eig(transition_matrix)
evectors = evectors.T
evectors = np.array([evectors[n] for n in range(len(evectors))])
idx = evalues.argsort()[::-1]
evalues = evalues[idx]
evectors = evectors[idx]

t_values = np.logspace(start=0, stop=5, num=1000)
eigengap_values = get_eigengap_values(t_values, evalues)
max_egap_vals, max_attained = get_maximal_eigengap_information(eigengap_values)

cluster_colours = make_colours(len(max_attained))

for idx, (n_clusters, info) in enumerate(max_attained.items()):
    colour = cluster_colours[idx]
    if n_clusters == 1:
        label = '1 Cluster Maxima'
    else:
        label = f'{n_clusters} Clusters Maxima'
    
    plt.axvline(
        x = info['n_steps'],
        color = colour,
        linestyle = '--',
        label = label
    )

plt.title('Gaussian Clouds Example 1\nMaximal Eigengap Separation')
plt.xlabel('Number of Steps')
plt.ylabel('Maximal Separation')
plt.legend()
plt.xscale('log')
plt.plot(t_values, list(max_egap_vals.values()), color='green')
plt.show()

In [ ]:
output = multiscale_k_prototypes(points, metric, similarity)

In [ ]:
for el in output:
    n_clusters = el['n_clusters']
    
    # skip one-cluster outputs
    if n_clusters == 1:
        continue
    
    suitability = int(100 * el['suitability']) / 100
    n_steps = el['suitability']
    partition = el['partition']
    
    cluster_colours = make_colours(n_clusters)
    
    for i, idx_set in enumerate(partition):
        colour = cluster_colours[i]
        cluster_x_coords = [x_coords[idx] for idx in idx_set]
        cluster_y_coords = [y_coords[idx] for idx in idx_set]
        sns.scatterplot(
            cluster_x_coords, 
            cluster_y_coords, 
            color = colour,
        )
        
    ax = plt.gca()
    ax.axes.xaxis.set_visible(False)
    ax.axes.yaxis.set_visible(False)
    
    plt.title(f'Gaussian Clouds Example 1\n{n_clusters} Clusters - Separation {suitability}')
    plt.show()

# Example 2

In [ ]:
points_filename = '../data/gaussian_clouds_2.pkl'

In [ ]:
with open(points_filename, 'rb') as fp:
    points = pickle.load(fp)

In [ ]:
x_coords = [el[0] for el in points]
y_coords = [el[1] for el in points]

In [ ]:
sns.scatterplot(x_coords, y_coords, color='purple')

ax = plt.gca()
ax.axes.xaxis.set_visible(False)
ax.axes.yaxis.set_visible(False)

plt.title('Gaussian Clouds Example 2\nUnclustered Points')
plt.show()

In [ ]:
MAX_CLUSTERS = 20

In [ ]:
metric = euclid_distance
distances = make_distances(points, metric)

flat_distances = [el for ls in distances for el in ls]
similarity = make_gaussian_similarity_from_percentile(flat_distances, 0.2)

W, D = make_W_D(points, metric, similarity)

transition_matrix = np.linalg.inv(D).dot(W)

evalues, evectors = np.linalg.eig(transition_matrix)
evectors = evectors.T
evectors = np.array([evectors[n] for n in range(len(evectors))])
idx = evalues.argsort()[::-1]
evalues = evalues[idx]
evectors = evectors[idx]

t_values = np.logspace(start=0, stop=8, num=1000)
eigengap_values = get_eigengap_values(t_values, evalues, MAX_CLUSTERS)
max_egap_vals, max_attained = get_maximal_eigengap_information(eigengap_values)

colours = make_colours(len(max_attained))

for idx, cluster in enumerate(max_attained):
    cluster_colour = colours[idx]
    cluster_egap_separation_values = list(eigengap_values[cluster].values())
    plt.plot(t_values, cluster_egap_separation_values, color=cluster_colour)
    maxima_value = max_attained[cluster]['suitability']
    maxima_location = max_attained[cluster]['n_steps']
    plt.axvline(
        x = maxima_location,
        color = cluster_colour,
        linestyle = '--',
        label = f'{cluster} Clusters Maxima'
    )

plt.ylim(0, 1.2)
plt.xscale('log')
plt.xlabel('Number of Steps')
plt.ylabel('Maximal Separation')
plt.legend()
plt.title(f'Gaussian Clouds Example 2\nMaximal Eigengap Separation')
plt.show()

In [ ]:
output = multiscale_k_prototypes(
    points,
    metric,
    similarity,
    max_clusters = MAX_CLUSTERS
)

In [ ]:
for el in output:
    n_clusters = el['n_clusters']
    
    # skip one-cluster outputs
    if n_clusters == 1:
        continue
    
    suitability = int(100 * el['suitability']) / 100
    n_steps = el['suitability']
    partition = el['partition']
    
    cluster_colours = make_colours(n_clusters)
    
    for i, idx_set in enumerate(partition):
        colour = cluster_colours[i]
        cluster_x_coords = [x_coords[idx] for idx in idx_set]
        cluster_y_coords = [y_coords[idx] for idx in idx_set]
        sns.scatterplot(
            cluster_x_coords, 
            cluster_y_coords, 
            color = colour,
        )
    
    ax = plt.gca()
    ax.axes.xaxis.set_visible(False)
    ax.axes.yaxis.set_visible(False)
    
    plt.title(f'{n_clusters} Clusters - Separation {suitability}')
    plt.title(f'Gaussian Clouds Example 2\n{n_clusters} Clusters - Separation {suitability}')
    plt.show()

Verify that point at the bottom-left has the largest distance to nearest neighbour of all points in the space

In [ ]:
leftmost_x_coord = min(x[0] for x in points)
leftmost_point = [x for x in points if np.isclose(x[0], leftmost_x_coord)][0]
other_points = [x for x in points if not np.isclose(x[0], leftmost_x_coord)]

In [ ]:
plt.scatter(leftmost_point[0], leftmost_point[1], color='red')
for other in other_points:
    plt.scatter(other[0], other[1], color='purple')
    
plt.title('Red - "bottom left" point, other points in purple')
plt.show()

In [ ]:
leftmost_point

Find, for each point, the distance to its nearest neighbour

In [ ]:
minimum_distances = {}

for pt in points:
    other_points = [
        point
        for point in points
        if not np.isclose(point[0], pt[0]) or not np.isclose(point[1], pt[1])
    ]
    distances = [metric(pt, other) for other in other_points]
    minimum_distances[pt[0]] = min(distances)

Find the maximum such distance and verify it belongs to the bottom-left point

In [ ]:
min_key = sorted(
    minimum_distances.keys(),
    key = lambda x: minimum_distances[x]
)[-1]

minimum_distances[min_key]

In [ ]:
full_points = points.copy()
reduced_points = [pt for pt in points if not np.isclose(pt[0], leftmost_point[0])]

In [ ]:
metric = euclid_distance

In [ ]:
full_distances = make_distances(full_points, metric)
reduced_distances = make_distances(reduced_points, metric)

full_flat_distances = [el for ls in full_distances for el in ls]
reduced_flat_distances = [el for ls in reduced_distances for el in ls]

full_similarity = make_gaussian_similarity_from_percentile(full_flat_distances, 0.2)
reduced_similarity = make_gaussian_similarity_from_percentile(reduced_flat_distances, 0.2)

full_W, full_D = make_W_D(full_points, metric, full_similarity)
reduced_W, reduced_D = make_W_D(reduced_points, metric, reduced_similarity)

full_transition_matrix = np.linalg.inv(full_D).dot(full_W)
reduced_transition_matrix = np.linalg.inv(reduced_D).dot(reduced_W)

full_evalues, full_evectors = np.linalg.eig(full_transition_matrix)
full_evectors = full_evectors.T
full_evectors = np.array([full_evectors[n] for n in range(len(full_evectors))])
idx = full_evalues.argsort()[::-1]
full_evalues = full_evalues[idx]
full_evectors = full_evectors[idx]

reduced_evalues, reduced_evectors = np.linalg.eig(reduced_transition_matrix)
reduced_evectors = reduced_evectors.T
reduced_evectors = np.array([reduced_evectors[n] for n in range(len(reduced_evectors))])
idx = reduced_evalues.argsort()[::-1]
reduced_evalues = reduced_evalues[idx]
reduced_evectors = reduced_evectors[idx]

In [ ]:
full_evalues[:15]

In [ ]:
full_evalues[:15] ** 6915

In [ ]:
reduced_evalues[:15]

In [ ]:
reduced_evalues[:15] ** 6915

In [ ]:
metric = euclid_distance
distances = make_distances(other_points, metric)

flat_distances = [el for ls in distances for el in ls]
similarity = make_gaussian_similarity_from_percentile(flat_distances, 0.2)

W, D = make_W_D(other_points, metric, similarity)

transition_matrix = np.linalg.inv(D).dot(W)

evalues, evectors = np.linalg.eig(transition_matrix)
evectors = evectors.T
evectors = np.array([evectors[n] for n in range(len(evectors))])
idx = evalues.argsort()[::-1]
evalues = evalues[idx]
evectors = evectors[idx]

t_values = np.logspace(start=0, stop=8, num=1000)
eigengap_values = get_eigengap_values(t_values, evalues, MAX_CLUSTERS)
max_egap_vals, max_attained = get_maximal_eigengap_information(eigengap_values)

colours = make_colours(len(max_attained))

for idx, cluster in enumerate(max_attained):
    cluster_colour = colours[idx]
    cluster_egap_separation_values = list(eigengap_values[cluster].values())
    plt.plot(t_values, cluster_egap_separation_values, color=cluster_colour)
    maxima_value = max_attained[cluster]['suitability']
    maxima_location = max_attained[cluster]['n_steps']
    plt.axvline(
        x = maxima_location,
        color = cluster_colour,
        linestyle = '--',
        label = f'{cluster} Clusters Maxima'
    )

plt.ylim(0, 1.2)
plt.xscale('log')
plt.xlabel('Number of Steps')
plt.ylabel('Maximal Separation')
plt.legend()
plt.title(f'Gaussian Clouds Example 2 - Point Removed\nMaximal Eigengap Separation')
plt.show()

In [ ]:
point_removed_output = multiscale_k_prototypes(
    reduced_points,
    metric,
    reduced_similarity,
    max_clusters = MAX_CLUSTERS
)

In [ ]:
point_removed_output

### Impact of the Similarity Function

In [ ]:
metric = euclid_distance
distances = make_distances(full_points, metric)

flat_distances = [el for ls in distances for el in ls]

similarity_functions = [
    ('inverse', lambda x: 1/x),
    ('inverse_squared', lambda x: 1/(x**2)),
    ('inverse_cubed', lambda x: 1/(x**3)),
    ('inverse_power_four', lambda x: 1/(x**4)),
    ('inverse_power_five', lambda x: (1/(x**5))),
    ('inverse_power_six', lambda x: (1/(x**6))),
    
]

for similarity_name, similarity in similarity_functions:
    W, D = make_W_D(other_points, metric, similarity)

    transition_matrix = np.linalg.inv(D).dot(W)

    evalues, evectors = np.linalg.eig(transition_matrix)
    evectors = evectors.T
    evectors = np.array([evectors[n] for n in range(len(evectors))])
    idx = evalues.argsort()[::-1]
    evalues = evalues[idx]
    evectors = evectors[idx]

    t_values = np.logspace(start=0, stop=8, num=1000)
    eigengap_values = get_eigengap_values(t_values, evalues, MAX_CLUSTERS)
    max_egap_vals, max_attained = get_maximal_eigengap_information(eigengap_values)

    colours = make_colours(len(max_attained))

    # plt.plot(t_values, list(max_egap_vals.values()), lw=2.5, color='green')

    for idx, cluster in enumerate(max_attained):
        cluster_colour = colours[idx]
        cluster_egap_separation_values = list(eigengap_values[cluster].values())
        plt.plot(t_values, cluster_egap_separation_values, color=cluster_colour)
        maxima_value = max_attained[cluster]['suitability']
        maxima_location = max_attained[cluster]['n_steps']
        plt.axvline(
            x = maxima_location,
            color = cluster_colour,
            linestyle = '--',
            label = f'{cluster} Clusters Maxima'
        )

    plt.ylim(0, 1.2)
    plt.xscale('log')
    plt.xlabel('Number of Steps')
    plt.ylabel('Maximal Separation')
    plt.legend()
    plt.title(f'Similarity {similarity_name}')
    
    plt.show()

In [ ]:
clusters = [1,2,3,4,5,8,15]
cluster_colours = make_colours(len(clusters))

custom_colours = {cluster: cluster_colours[idx] for idx, cluster in enumerate(clusters)}

In [ ]:
metric = euclid_distance
distances = make_distances(full_points, metric)

flat_distances = [el for ls in distances for el in ls]

similarity = make_gaussian_similarity_from_percentile(flat_distances, 0.2)

similarity_percentiles = [0.2, 0.5]

for percentile in similarity_percentiles:
    similarity_sigma = np.percentile(flat_distances, percentile)
    similarity = make_gaussian_similarity(similarity_sigma)
    
    print(f'percentile = {percentile}')
    print(f'sigma = {similarity_sigma}')

    W, D = make_W_D(other_points, metric, similarity)

    transition_matrix = np.linalg.inv(D).dot(W)

    evalues, evectors = np.linalg.eig(transition_matrix)
    evectors = evectors.T
    evectors = np.array([evectors[n] for n in range(len(evectors))])
    idx = evalues.argsort()[::-1]
    evalues = evalues[idx]
    evectors = evectors[idx]

    t_values = np.logspace(start=0, stop=11, num=1000)
    eigengap_values = get_eigengap_values(t_values, evalues, MAX_CLUSTERS)
    max_egap_vals, max_attained = get_maximal_eigengap_information(eigengap_values)

    colours = make_colours(len(max_attained))

    for idx, cluster in enumerate(max_attained):
        cluster_colour = colours[idx]
        cluster_egap_separation_values = list(eigengap_values[cluster].values())
        plt.plot(t_values, cluster_egap_separation_values, color=cluster_colour)
        maxima_value = max_attained[cluster]['suitability']
        maxima_location = max_attained[cluster]['n_steps']
        plt.axvline(
            x = maxima_location,
            color = cluster_colour,
            linestyle = '--',
            label = f'{cluster} Clusters Maxima'
        )
        
    title_sigma = int(1000*similarity_sigma) / 1000

    plt.ylim(0, 1.2)
    plt.xscale('log')
    plt.xlabel('Number of Steps')
    plt.ylabel('Maximal Separation')
    plt.legend()
    plt.title(f'Gaussian Clouds Example 2\nImpact of the Similarity Function\n$\sigma = {title_sigma}$')
    
    percentile_str = str(percentile).replace('.', '_pt_')
    plt.show()